In [1]:
import os
import warnings

# Suppress tqdm IProgress warning in Jupyter (optional: pip install ipywidgets for widget progress bars)
try:
    from tqdm import TqdmWarning
    warnings.filterwarnings("ignore", category=TqdmWarning)
except ImportError:
    pass

import pandas as pd
from openai import OpenAI

In [2]:
# read case data from csv file
case_data = pd.read_csv('data/dummy_data_clean.csv')

In [3]:
# convert case_data.case_text_lower to Kura Conversation object

from kura.types import Message, Conversation
from datetime import datetime
from rich import print


def process_case_row(row) -> Conversation:
    """Build a Kura Conversation from a case_data row (case_id, case_text_lower)."""
    case_id = row["case_id"]
    case_text = row["case_text_lower"] if pd.notna(row["case_text_lower"]) else ""
    return Conversation(
        chat_id=str(case_id),
        created_at=datetime.now(),
        messages=[
            Message(
                created_at=datetime.now(), # replace with case_data.created_at
                # product = case_data.product
                role="user",
                content=case_text,
            )
        ],
        metadata={"case_id": case_id},
    )


# Build conversations from case_data and show the first one
conversations = [process_case_row(case_data.iloc[i]) for i in range(len(case_data))]
print(conversations[0])


Conversation(
    chat_id='65047504',
    created_at=datetime.datetime(2026, 2, 18, 14, 49, 17, 59309),
    messages=[
        Message(
            created_at=datetime.datetime(2026, 2, 18, 14, 49, 17, 59316),
            role='user',
            content='product: time & labor | case type: support requested - time & labor | case detail: time & 
labor | subject: holiday | description: helped the client with the holiday list'
        )
    ],
    metadata={'case_id': 65047504.0}
)

In [4]:
from kura.checkpoints import JSONLCheckpointManager
from kura.summarisation import summarise_conversations, SummaryModel, DEFAULT_SUMMARY_PROMPT
from kura.types.summarisation import GeneratedSummary
from kura.cluster import generate_base_clusters_from_conversation_summaries, ClusterDescriptionModel
from kura.meta_cluster import reduce_clusters_from_base_clusters, MetaClusterModel
from kura.dimensionality import reduce_dimensionality_from_clusters, HDBUMAP
from kura.visualization import visualise_pipeline_results




In [6]:
# Model setup from config: switch between OpenAI / Bedrock (summary) and OpenAI / local (embeddings)
import config
from config import (
    SUMMARIZATION_BACKEND,
    SUMMARIZATION_MODEL,
    EMBEDDING_BACKEND,
    EMBEDDING_MODEL,
    get_checkpoint_dir,
)
from kura.embedding import OpenAIEmbeddingModel, SentenceTransformerEmbeddingModel

CHECKPOINT_DIRECTORY = str(get_checkpoint_dir())


def get_summary_model():
    """Build SummaryModel from config. Use OpenAI or AWS Bedrock via Instructor provider string."""
    # Instructor format: "provider/model-id"
    # OpenAI:   "openai/gpt-4o-mini", "openai/gpt-4o"
    # Bedrock:  "bedrock/anthropic.claude-3-5-sonnet-20241022-v2:0" (set AWS credentials / region)
    if SUMMARIZATION_BACKEND == "bedrock":
        model_id = f"bedrock/{SUMMARIZATION_MODEL}"
    else:
        model_id = f"openai/{SUMMARIZATION_MODEL}"
    return SummaryModel(model=model_id, max_concurrent_requests=10)


def get_embedding_model():
    """Build embedding model from config. Use OpenAI API or local SentenceTransformer (e.g. all-mpnet-base-v2)."""
    if EMBEDDING_BACKEND == "local":
        # Local: sentence-transformers (pip install sentence-transformers)
        # Examples: "all-mpnet-base-v2", "all-MiniLM-L6-v2"
        return SentenceTransformerEmbeddingModel(
            model_name=EMBEDDING_MODEL,
            model_batch_size=128,
            device="cpu",  # or "cuda" if GPU
        )
    else:
        # OpenAI: "text-embedding-3-small", "text-embedding-3-large"
        return OpenAIEmbeddingModel(
            model_name=EMBEDDING_MODEL,
            model_batch_size=50,
            n_concurrent_jobs=5,
        )


# Optional: print active config
print(f"Summary: {SUMMARIZATION_BACKEND} / {SUMMARIZATION_MODEL}")
print(f"Embedding: {EMBEDDING_BACKEND} / {EMBEDDING_MODEL}")
print(f"Checkpoints: {CHECKPOINT_DIRECTORY}")

Summary: openai / gpt-4o-mini

Embedding: local / all-mpnet-base-v2

Checkpoints: /Users/Dev/Documents/coding/case-clustering-kura/checkpoints

In [8]:
async def analyze_conversations(conversations, checkpoint_manager):
    
    # Set up models
    summary_model = SummaryModel()
    cluster_model = ClusterDescriptionModel()
    meta_cluster_model = MetaClusterModel()
    dimensionality_model = HDBUMAP()

    # Run pipeline steps
    summaries = await summarise_conversations(
        conversations, model=summary_model, checkpoint_manager=checkpoint_manager
    )

    clusters = await generate_base_clusters_from_conversation_summaries(
        summaries, model=cluster_model, checkpoint_manager=checkpoint_manager
    )

    reduced_clusters = await reduce_clusters_from_base_clusters(
        clusters, model=meta_cluster_model, checkpoint_manager=checkpoint_manager
    )

    projected = await reduce_dimensionality_from_clusters(
        reduced_clusters,
        model=dimensionality_model,
        checkpoint_manager=checkpoint_manager,
    )

    return projected

In [9]:
# run the pipeline
checkpoint_manager = JSONLCheckpointManager(CHECKPOINT_DIRECTORY, enabled=True)
checkpoint_manager.save_checkpoint("conversations", conversations)
clusters = await analyze_conversations(
    conversations, checkpoint_manager=checkpoint_manager
)

Summarising 48 conversations: 100%|██████████| 48/48 [00:04<00:00, 10.04it/s]


In [10]:
# Get top-level clusters (those without parents)
parent_clusters = [cluster for cluster in clusters if cluster.parent_id is None]

# Format each cluster's info with name, description and number of chats
formatted_clusters = []
for cluster in parent_clusters:
    cluster_info = (
        f"[bold]{cluster.name}[/bold] : {cluster.description} : {len(cluster.chat_ids)}"
    )
    formatted_clusters.append(cluster_info)

# Join with newlines and print
print("\n\n".join(formatted_clusters))

Assist with HR and payroll issues : The user sought assistance with various HR and payroll-related issues, 
including employee transfers, tax code updates, and access problems. They requested guidance on resolving these 
issues and received support for their inquiries. : 12

Clarify tax rate changes for unemployment insurance : The users inquired about various discrepancies and updates 
related to tax rate changes, particularly focusing on state unemployment insurance rates. They sought confirmation,
clarification, and assistance with recalculations and documentation regarding these changes. : 8

Assist with time and labor system inquiries : The user sought assistance with various inquiries related to time and
labor systems, including payroll policies, employee access issues, and system outages. They received guidance on 
managing holiday lists, supervisor visibility, and payroll reports, indicating a focus on operational support 
within HR functions. : 16

Assist with payroll submission and check management : The user sought assistance regarding payroll submission 
deadlines and check management processes, including issuing stop payments and voiding checks. They clarified 
specific dates and procedures to ensure compliance with payroll requirements. : 6

Assist with tax notice and rate change inquiries : The user sought assistance with various tax-related inquiries, 
including tax notices, rate changes, and payroll tax credits. They requested guidance on resolving issues related 
to tax rates and documentation for specific cases. : 6

In [11]:
# Analysing Our Summaries
from kura.types import ConversationSummary
from kura.checkpoints import JSONLCheckpointManager

checkpoint_manager = JSONLCheckpointManager(CHECKPOINT_DIRECTORY, enabled=True)
summaries = checkpoint_manager.load_checkpoint("summaries", ConversationSummary)
conversations = checkpoint_manager.load_checkpoint("conversations", Conversation)


id_to_conversation = {
    conversation.chat_id: conversation for conversation in conversations
}


for i in range(10):
    print(summaries[i].summary)
    print(id_to_conversation[summaries[i].chat_id].messages[0].content)

The user requested assistance with a holiday list related to time and labor support. They needed help in organizing
or managing this list for a client.

product: time & labor | case type: support requested - time & labor | case detail: time & labor | subject: holiday 
| description: helped the client with the holiday list

The user needed guidance on why a supervisor could not see all direct reports and was directed to review employee 
configuration for errors in supervisor setup.

product: hr & payroll | case type: 00 - how to/self resolved | case detail: how to/self resolved | subject: time & 
labor groups | description: client needed guidance as to why their supervisor could not see all of their direct 
reports. guided them to review the employee configuration and notices errors in supervisor setup

The user inquired about payroll submission deadlines and whether they could submit payroll on a specific date, 
while also seeking assistance regarding an ongoing project.

...

product: tax account maintenance | case type: tax rate change | case detail: sui rate change | subject: florida 
2023 tax rate | description: 

hello!

please see xxxxx florida 2023 tax rate. thank you!


thank you,

The user is inquiring about payroll policies related to time and labor support. They have submitted a case 
regarding this topic.

product: time & labor | case type: support requested - time & labor | case detail: time & labor | subject: payroll 
policies | description: payroll policies

The user is reporting a system outage and advises to wait for 15 minutes. The case is categorized under HR and 
payroll with a self-resolved type.

product: hr & payroll | case type: 00 - how to/self resolved | case detail: how to/self resolved | subject: system 
outage | description: adv er to wait 15 min

In [12]:
from kura.visualization import visualise_pipeline_results

# Minimal tree
visualise_pipeline_results(clusters, style="basic")

# Default: tree + percentages + progress bars + stats
#visualise_pipeline_results(clusters, style="enhanced")

# Colored + tables (needs: pip install rich)
#visualise_pipeline_results(clusters, style="rich")

Clusters (48 conversations)
╠══ Assist with HR and payroll issues (12 conversations)
╠══ Clarify tax rate changes for unemployment insurance (8 conversations)
╠══ Assist with time and labor system inquiries (16 conversations)
╠══ Assist with payroll submission and check management (6 conversations)
╚══ Assist with tax notice and rate change inquiries (6 conversations)



In [ ]:
# 